In [2]:
from core import isb, database
import pandas as pd
import datetime

In [3]:
db = database()

C:\Users\palmarg\AppData\Roaming\Python\Python312\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
df = db.query("""
select 
  r.realestate_id, 
  rt.name as realestate_type_name, 
  sa.agreement_date, 
  sa.registration_date, 
  sa.purchase_price, 
  is_valid_agreement, 
  is_fully_complete, 
  r.square_meters, 
  construction_year, 
  c.name as city_name, 
  coalesce((select True from listings lst where lst.realestate_id = r.realestate_id and lst.span && daterange((sa.registration_date + interval '1' month)::date, (sa.registration_date + interval '6' month)::date, '[]') limit 1), False) as listed_after_sale,
  count(1) over (partition by r.realestate_id order by registration_date) as sale_counter
from sale_agreements sa
join realestates r
on sa.realestate_id = r.realestate_id
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands l
on a.land_id = l.land_id
join postals p
on l.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'      
""")

In [5]:
df['registration_year'] = df['registration_date'].pipe(pd.to_datetime).dt.year
df['registration_day_of_year'] = df['registration_date'].pipe(pd.to_datetime).dt.dayofyear

In [13]:
df_total = db.query("""
select coalesce(construction_year, 0) as year, count(1) as total_units
from realestates r
join realestate_types rt
on r.realestate_type_id = rt.realestate_type_id
join units u 
on r.unit_id = u.unit_id
join addresses a
on u.address_id = a.address_id
join lands l
on a.land_id = l.land_id
join postals p
on l.postal_id = p.postal_id
join cities c
on p.city_id = c.city_id
join regions reg
on c.region_id = reg.region_id
where rt.name in ('Fjölbýlishús', 'Par/Raðhús', 'Hæð', 'Einbýlishús')
  and reg.name = 'Höfuðborgarsvæðið'
group by coalesce(construction_year, 0)        
""")

In [7]:
def make_plot(df, annotate_shift_y=True, show_only_year=None, subtitle='Samanburður síðustu 10 ár á uppsafnaðri sölu innan árs á höfuðborgarsvæðinu.<br>Hlutfallsleg sala er fengin með því að deila fjölda gildra kaupsamninga með fjölda íbúða í upphafi hvers árs.'):

    fig = isb.Figure('core')

    for i, column in enumerate(df.columns):

        if show_only_year and show_only_year != column:
            continue

        fig.add_scatter(
            x = df.index,
            y = df[column],
            line = dict(
                color = fig.colors(0) if column == 2025 else fig.font_color.alpha(0.5),
                width = 3 if column == 2025 else 1
            ),
            showlegend = False,
            name=column
        )

    row = df.tail(1).iloc[0].sort_values()

    for i, (key, value) in enumerate(row.items()):
        
        if show_only_year and show_only_year != key:
            continue

        fig.add_annotation(
            axref='x',
            ayref='y',
            x = row.name,
            y = value,
            ax = datetime.date(2025,11,28),
            ay = (0.02 + (0.06 * (i/8))) if annotate_shift_y else value,
            font = fig.get_font(size=16),
            text = fig.color_string(f'<color 0><b>{key}</b></color>: {value:.2%}') if key == 2025 else f'<b>{key}</b>: {value:.2%}',
            xanchor = 'left',
            align = 'center',
            bgcolor = fig.bg_color,
            arrowhead=0,
            arrowcolor=fig.font_color.alpha(.6),
            arrowwidth = .5
        )

    for i, column in enumerate(df.columns):
        
        if show_only_year and show_only_year != column:
            continue

        fig.add_scatter(
            x = [row.name],
            y = [row[column]],
            marker = dict(
                size = 8,
                color = fig.colors(0) if column == 2025 else fig.font_color.alpha(0.5),
            ),
            showlegend=False
        )
        fig.add_scatter(
            x = [row.name],
            y = [row[column]],
            marker = dict(
                size = 4,
                color = fig.bg_color
            ),
            showlegend=False
        )

    fig.update_yaxes(
        tickformat = '.1%',
        #range = [0, 0.085]
    )
    fig.update_xaxes(
        tickformat = "%b"
    )
    fig.set_title(
        "Uppsöfnuð hlutfallsleg sala á fasteignamarkaði",
       )

    fig.add_annotation(
        x = -0.027,
        y = 1.13,
        showarrow=False,
        font = fig.get_font(size=18),
        text = f"<i>{subtitle}</i>",
        xref='paper', yref='paper',
        xanchor='left',
        align='left'

    )

    fig.update_layout(
        margin_t = 150,
        #title_y = 0.93
    )

    #fig.add_logo()

    return fig




df_temp = df\
.loc[lambda r: r.is_valid_agreement]\
.pivot_table(index='registration_day_of_year', columns='registration_year', values='realestate_type_name', aggfunc='count')\
.reindex(range(0, 367))\
.fillna(0)\
.cumsum()\
.reset_index()\
.melt(id_vars='registration_day_of_year', var_name='year', value_name='sales')\
.merge(
    df_total\
    .set_index('year')\
    .cumsum()\
    .reset_index(),
    on='year'
)\
.assign(turnover_rate = lambda r: r.sales / r.total_units)\
.loc[lambda r: r.year >= 2016]\
.pivot(index='registration_day_of_year', columns='year', values='turnover_rate')\
.loc[lambda r: r.index <= 318]\
.reset_index()\
.assign(date = lambda r: r.registration_day_of_year.apply(lambda x: datetime.date(2025,1,1) + datetime.timedelta(days=x)))\
.set_index('date')\
.drop(['registration_day_of_year'], axis=1)\
.rolling(3)\
.mean()


df_temp\
.pipe(make_plot, show_only_year=2025)\
.export("relative-sale-amount--only-2025", scale=1.5)


df_temp\
.pipe(make_plot, show_only_year=None)\
.export("relative-sale-amount", scale=1.5)

In [8]:
years = []
for i, year in enumerate([2025, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]):

    years.append(year)

    df\
    .loc[lambda r: r.is_valid_agreement]\
    .pivot_table(index='registration_day_of_year', columns='registration_year', values='realestate_type_name', aggfunc='count')\
    .reindex(range(0, 367))\
    .fillna(0)\
    .cumsum()\
    .reset_index()\
    .melt(id_vars='registration_day_of_year', var_name='year', value_name='sales')\
    .merge(
        df_total\
        .set_index('year')\
        .cumsum()\
        .reset_index(),
        on='year'
    )\
    .assign(turnover_rate = lambda r: r.sales / r.total_units)\
    .loc[lambda r: r.year.isin(years)]\
    .pivot(index='registration_day_of_year', columns='year', values='turnover_rate')\
    .loc[lambda r: r.index <= 318]\
    .reset_index()\
    .assign(date = lambda r: r.registration_day_of_year.apply(lambda x: datetime.date(2025,1,1) + datetime.timedelta(days=x)))\
    .set_index('date')\
    .drop(['registration_day_of_year'], axis=1)\
    .rolling(3)\
    .mean()\
    .pipe(make_plot, annotate_shift_y=False, subtitle="Samanburður ára fyrir hrun á uppsafnaðri sölu innan árs á höfuðborgarsvæðinu.<br>Hlutfallsleg sala er fengin með því að deila fjölda gildra kaupsamninga með fjölda íbúða í upphafi hvers árs")\
    .export(f"03-{i}-relative-sale-amount", scale=1.5)

In [9]:
df_total

,year,total_units
0,0,701
1,1841,4
2,1848,1
3,1850,3
4,1858,2
...,...,...
152,2021,1745
153,2022,1630
154,2023,2148
155,2024,2035
